In [1]:
%load_ext autoreload
%autoreload 2
from src.data_collection import *
from tqdm import tqdm
import numpy as np

# Create setup:

In [2]:
import os
os.makedirs("data/raw", exist_ok=True)
os.makedirs("data/clean", exist_ok=True)
os.makedirs("data/compressed", exist_ok=True)

# Data scraping from Wikipedia lists:

In [71]:
urls = [
"https://en.wikipedia.org/wiki/List_of_fiction_works_made_into_feature_films_(0-9,_A-C)",
"https://en.wikipedia.org/wiki/List_of_fiction_works_made_into_feature_films_(D-J)",
"https://en.wikipedia.org/wiki/List_of_fiction_works_made_into_feature_films_(K-R)",
"https://en.wikipedia.org/wiki/List_of_fiction_works_made_into_feature_films_(S-Z)",
"http://en.wikipedia.org/wiki/List_of_short_fiction_made_into_feature_films",
"https://en.wikipedia.org/wiki/List_of_children%27s_books_made_into_feature_films",
#"https://en.wikipedia.org/wiki/List_of_films_based_on_comics"
]
df = scrape_wikipedia_list(urls)
df.columns=['Book','author_book','Film',"Year_book", "Year_film"]

df # fix later


,Book,author_book,Film,Year_book,Year_film
0,The 25th Hour,David Benioff,25th Hour,2001,2002
1,3 Assassins,Kōtarō Isaka,Grasshopper,2004,2015
2,4.50 from Paddington,Agatha Christie,"Murder, She Said",1957,1961
3,4.50 from Paddington,Agatha Christie,Crime Is Our Business,1957,2008
4,58 Minutes,Walter Wager,Die Hard 2,1987,1990
...,...,...,...,...,...
5805,Z for Zachariah,Robert C. O'Brien,Z for Zachariah,1974,2015
5806,Zathura,Chris Van Allsburg,Zathura,2002,2005
5807,Zenon: Girl of the 21st Century,Marilyn Sadler,Zenon: Girl of the 21st Century,1997,1999
5808,Zenon: Girl of the 21st Century,Marilyn Sadler,Zenon: The Zequel,1997,2001


In [75]:
r = tmdb_search_movie("zathura")
r

{'adult': False,
 'backdrop_path': '/s9Bn7Mx4U9cWOR1WdFZjU1DTAkh.jpg',
 'genre_ids': [878, 12, 10751],
 'id': 6795,
 'original_language': 'en',
 'original_title': 'Zathura: A Space Adventure',
 'overview': 'After their father is called into work, two young boys, Walter and Danny, are left in the care of their teenage sister, Lisa, and told they must stay inside. Walter and Danny, who anticipate a boring day, are shocked when they begin playing Zathura, a space-themed board game, which they realize has mystical powers when their house is shot into space. With the help of an astronaut, the boys attempt to return home.',
 'popularity': 6.1157,
 'poster_path': '/gDb5BW2NLqZ9cvg9nyzkZVvmgze.jpg',
 'release_date': '2005-11-06',
 'title': 'Zathura: A Space Adventure',
 'video': False,
 'vote_average': 6.43,
 'vote_count': 3438}

# Scraping tmdb data:

In [74]:
data = {
    "Book" : [],
    "author_book": [],
    "Film" : [],
    "date_film" : [],
    "votes_film" : [],
    "overview_film" : [],
    "vote_count_film" : [],
    "popularity_film" : []
}
for i in tqdm(range(len(df))):
    n = df["Film"][i]
    y = df["Year_film"][i]
    k = tmdb_search_movie(n,year=y)
    if k is not None: 
        data["votes_film"].append(k["vote_average"])
        data["Film"].append(k["original_title"])
        data["date_film"].append(k["release_date"])
        data["overview_film"].append(k["overview"])        
        data["vote_count_film"].append(k["vote_count"])
        data["popularity_film"].append(k["popularity"])

    else: 
        data["votes_film"].append(np.nan)
        data["Film"].append(n)
        data["date_film"].append(str(y))
        data["overview_film"].append(np.nan)
        data["vote_count_film"].append(np.nan)
        data["popularity_film"].append(np.nan)
    data["Book"].append(df["Book"][i])
    data["author_book"].append(df["author_book"][i])
df = pd.DataFrame(data)
df.to_csv("data/raw/movie_adaptation_list.csv")
df

100%|███████████████████████████████████████████| 5810/5810 [05:55<00:00, 16.32it/s]


,Book,author_book,Film,date_film,votes_film,overview_film,vote_count_film,popularity_film
0,The 25th Hour,David Benioff,25th Hour,2002-12-19,7.322,In New York City in the days following the eve...,2462.0,3.4212
1,3 Assassins,Kōtarō Isaka,グラスホッパー,2015-11-07,5.800,Movie contains the stories of three people: Su...,13.0,3.6855
2,4.50 from Paddington,Agatha Christie,Murder She Said,1961-09-26,7.304,Miss Marple believes she's seen a murder in a ...,219.0,2.5151
3,4.50 from Paddington,Agatha Christie,Le crime est notre affaire,2008-10-02,5.883,"Loosely based on ""4.50 from Paddington"" by Aga...",94.0,1.5432
4,58 Minutes,Walter Wager,Die Hard 2,1990-07-03,6.960,"One year after his heroics in Los Angeles, Joh...",6133.0,7.0722
...,...,...,...,...,...,...,...,...
5805,Z for Zachariah,Robert C. O'Brien,Z for Zachariah,2015-08-13,5.687,"In the wake of a nuclear war, a young woman su...",966.0,1.6881
5806,Zathura,Chris Van Allsburg,Zathura: A Space Adventure,2005-11-06,6.430,"After their father is called into work, two yo...",3438.0,6.1157
5807,Zenon: Girl of the 21st Century,Marilyn Sadler,Zenon: Girl of the 21st Century,1999-01-23,6.049,"Zenon Kar, a 13-year-old girl who lives on a s...",183.0,1.5327
5808,Zenon: Girl of the 21st Century,Marilyn Sadler,Zenon: The Zequel,2001-01-12,5.600,Zenon Kar is 15 and lives on a space station w...,108.0,2.4229


In [ ]:
import json
df2 = pd.read_csv("data/raw/booksummaries.csv",sep="\t")
df2.columns=["a1","a2","Book","Author","year","Genre","overview"]
df2 = df2.drop(columns=["a1","a2"])
genres = []
for i in df2["Genre"]:
    if isinstance(i,str):
        genres.append(list(json.loads(i).values()))
    else:
        genres.append(np.nan)
df2["Genre"] = genres
df2

In [53]:
import numpy as np
count_nan_year = (df2["year"].isnull()).sum().sum()
count_nan_genre = (df2["Genre"].isnull()).sum().sum()
count_nan_overview = (df2["overview"].isnull()).sum().sum()

print(f'{count_nan_year =}')
print(f'{count_nan_genre =}')
print(f'{count_nan_overview =}')


count_nan_year =np.int64(5610)
count_nan_genre =np.int64(3718)
count_nan_overview =np.int64(0)
